In [41]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

### 合并数据集方便处理

In [42]:
path = 'E:/Source/ThsisSource/DataSource/CICIDS2017/data/'
save_path = 'E:/Source/ThsisSource/DataSource/CICIDS2017/process/'
csv_file =['Friday-WorkingHours-Afternoon-DDos.pcap_ISCX','Friday-WorkingHours-Morning.pcap_ISCX',
'Monday-WorkingHours.pcap_ISCX','Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX',
'Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX','Tuesday-WorkingHours.pcap_ISCX','Wednesday-workingHours.pcap_ISCX']
end = '.csv'
data = pd.read_csv(path+csv_file[0]+end)
# 循环添加数据集
for i in range(1,7):
    tmp_data=pd.read_csv(path+csv_file[i]+end)
    data=pd.concat([data,tmp_data],axis=0,ignore_index=True)
data

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,54865,3,2,0,12,0,6,6,6.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,55054,109,1,1,6,6,6,6,6.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,55055,52,1,1,6,6,6,6,6.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,46236,34,1,1,6,6,6,6,6.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,54863,3,2,0,12,0,6,6,6.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2544271,53,32215,4,2,112,152,28,28,28.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2544272,53,324,2,2,84,362,42,42,42.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2544273,58030,82,2,1,31,6,31,0,15.5,21.92031,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2544274,53,1048635,6,2,192,256,32,32,32.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


In [43]:
# 获取特征行
data.keys()

Index([' Destination Port', ' Flow Duration', ' Total Fwd Packets',
       ' Total Backward Packets', 'Total Length of Fwd Packets',
       ' Total Length of Bwd Packets', ' Fwd Packet Length Max',
       ' Fwd Packet Length Min', ' Fwd Packet Length Mean',
       ' Fwd Packet Length Std', 'Bwd Packet Length Max',
       ' Bwd Packet Length Min', ' Bwd Packet Length Mean',
       ' Bwd Packet Length Std', 'Flow Bytes/s', ' Flow Packets/s',
       ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min',
       'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max',
       ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Std',
       ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags', ' Bwd PSH Flags',
       ' Fwd URG Flags', ' Bwd URG Flags', ' Fwd Header Length',
       ' Bwd Header Length', 'Fwd Packets/s', ' Bwd Packets/s',
       ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean',
       ' Packet Length Std', ' Packet Length Variance', '

### 对数据集中的缺失值补0

In [44]:
data = data.fillna(value=0)

### 对数据集中Flow Packets/s Flow Bytes/s的无穷大值进行修改

In [45]:
for i in range(len(data[' Flow Packets/s'])):
    if data[' Flow Packets/s'][i] == float('inf'):
        data.loc[i, ' Flow Packets/s'] = '2000001'
for i in range(len(data['Flow Bytes/s'])):
    if data['Flow Bytes/s'][i] == float('inf'):
        data.loc[i, 'Flow Bytes/s'] = '1040000001'

### 将Label改成数字类型

In [46]:
Label = data[' Label']
Label = np.array(Label)
# 删除标签列
data_ = data.drop([' Label'], axis=1)
data = np.array(data_)
# 修改
i = 0
for i in range(Label.shape[0]):
    if Label[i] == "BENIGN":
        Label[i] = 0
    elif Label[i] == "DoS Hulk":
        Label[i] = 1
    elif Label[i] == "PortScan":
        Label[i] = 2
    elif Label[i] == "DDoS":
        Label[i] = 3
    elif Label[i] == "DoS GoldenEye":
        Label[i] = 4
    elif Label[i] == "FTP-Patator":
        Label[i] = 5
    elif Label[i] == "SSH-Patator":
        Label[i] = 6
    elif Label[i] == "DoS slowloris":
        Label[i] = 7
    elif Label[i] == "DoS Slowhttptest":
        Label[i] = 8
    elif Label[i] == "Bot":
        Label[i] = 9
    elif Label[i] == "Web Attack � Brute Force":
        Label[i] = 10
    elif Label[i] == "Web Attack � XSS":
        Label[i] = 11
    elif Label[i] == "Infiltration":
        Label[i] = 12
    elif Label[i] == "Web Attack � Sql Injection":
        Label[i] = 13
    elif Label[i] == "Heartbleed":
        Label[i] = 14
Label

array([0, 0, 0, ..., 0, 0, 0], dtype=object)

In [47]:
pd.value_counts(Label)

0     2145560
1      231073
3      128027
4       10293
5        7938
6        5897
7        5796
8        5499
9        1966
10       1507
11        652
12         36
13         21
14         11
dtype: int64

### 层次划分数据集

In [48]:
#为了缩小数据集同时不改变少数类的样本过少
train_x,test_x,train_y,test_y = train_test_split(data,Label,test_size=0.01,random_state=50,stratify=Label)
pd.value_counts(pd.DataFrame(test_y)[0])

0     21456
1      2311
3      1280
4       103
5        79
6        59
7        58
8        55
9        20
10       15
11        7
Name: 0, dtype: int64

### 数据标准化和数据降维


In [49]:
# 标准化降维
X = test_x.astype(float)
Y = test_y.reshape(-1,1)
# 标准化
scaler = StandardScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)
# 降维
pca = PCA(n_components=0.9)
pca.fit(X_scaled)
X_pca = pca.transform(X_scaled)
# # 拼接
X_pca = np.array(X_pca)
X2 = np.concatenate((X_pca, Y), axis=1)
print(X2.shape)
data2 = pd.DataFrame(X2)
data2

(25443, 19)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,-1.961457,0.115972,0.152237,-0.539005,1.397317,-0.767677,-0.587526,0.68422,0.314444,0.044094,-0.03503,-0.269077,-0.352243,0.009447,-0.321033,0.371283,-0.434435,0.219216,0
1,-1.922332,0.11435,0.171017,-0.426571,1.045668,-0.492688,-0.554752,0.28382,0.106765,0.033881,-0.030989,0.03437,0.068615,0.062913,0.327737,-0.607207,0.431509,-0.243561,0
2,-1.92039,0.116021,0.149668,-0.458289,1.28508,-0.700127,-0.627095,0.523537,0.200675,0.053202,-0.07355,0.009436,0.046475,0.089891,0.225861,-0.590866,0.454608,-0.236969,0
3,6.502417,-0.477917,-9.382772,-2.839459,0.135395,-1.364041,4.154565,1.174539,-0.20439,-0.100944,-0.46612,1.775337,1.920555,-0.024892,0.494611,-1.306132,-0.598472,0.120916,3
4,-1.275265,0.026733,0.659537,-0.393359,-0.789874,0.309302,-0.469766,-0.381573,-0.461201,0.049592,0.11778,-0.47676,-0.695294,-0.536346,0.647509,-0.890011,0.514954,-0.685243,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25438,-1.368477,0.039868,0.930685,-0.521185,1.414958,-0.771799,-0.076458,0.462415,0.13435,0.043187,-0.120419,0.259878,0.132078,-0.050571,0.252696,-0.646665,0.413783,-0.19476,0
25439,-1.861606,0.11116,0.04834,-0.642456,1.490045,-0.841478,-0.692223,0.740889,0.301403,0.072214,-0.125639,0.029577,0.007108,0.070334,0.119261,-0.621526,0.46316,-0.14562,0
25440,-2.196824,0.137767,0.371273,0.466872,-1.903179,0.493647,-0.050331,-1.216475,-3.292414,-0.309208,1.910641,1.445802,1.281378,0.193889,-0.476502,1.008032,3.555716,-0.246856,0
25441,-2.209588,0.124097,0.500756,0.079017,-2.889,0.55684,-0.081561,-0.113455,0.451542,0.133222,-1.029969,1.404797,-1.458095,-0.210387,-0.436866,-0.07809,-1.227589,-0.056121,0


### 保存文件到对应路径

In [50]:
data2.to_csv(save_path+'result_10checkpoint'+end, encoding='utf-8')